In [12]:
# ----------------------------------------------------------------------
# Numenta Platform for Intelligent Computing (NuPIC)
# Copyright (C) 2019, Numenta, Inc.  Unless you have an agreement
# with Numenta, Inc., for a separate license for this software code, the
# following terms and conditions apply:
#
# This program is free software: 
# you can redistribute it and/or modify
# it under the terms of the GNU Affero Public License version 3 as
# published by the Free Software Foundation.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Affero Public License for more details.
#
# You should have received a copy of the GNU Affero Public License
# along with this program.  If not, see http://www.gnu.org/licenses.
#
# http://numenta.org/licenses/
# ----------------------------------------------------------------------

In [13]:
!pip install git+https://github.com/numenta/nupic.torch.git#egg=nupic.torch
!pip install torch torchvision

  Cloning https://github.com/numenta/nupic.torch.git to /private/var/folders/0k/nx51st811lj2bkn32z3c0_dc0000gn/T/pip-install-h0b3r322/nupic-torch_6389283cd1294fb79c8a7c3c4e32176d
  Running command git clone --filter=blob:none --quiet https://github.com/numenta/nupic.torch.git /private/var/folders/0k/nx51st811lj2bkn32z3c0_dc0000gn/T/pip-install-h0b3r322/nupic-torch_6389283cd1294fb79c8a7c3c4e32176d
  Resolved https://github.com/numenta/nupic.torch.git to commit 51269b7aae1024ee301837320a3b3ebee538c8c0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm

SEED = 18
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
def train(model, loader, optimizer, criterion, post_batch_callback=None):
    """
    Train the model using given dataset loader. 
    Called on every epoch.
    :param model: pytorch model to be trained
    :type model: torch.nn.Module
    :param loader: dataloader configured for the epoch.
    :type loader: :class:`torch.utils.data.DataLoader`
    :param optimizer: Optimizer object used to train the model.
    :type optimizer: :class:`torch.optim.Optimizer`
    :param criterion: loss function to use
    :type criterion: function
    :param post_batch_callback: function(model) to call after every batch
    :type post_batch_callback: function
    """
    model.train()
    for batch_idx, (data, target) in enumerate(tqdm(loader, leave=False)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if post_batch_callback is not None:
            post_batch_callback(model)


def test(model, loader, criterion):
    """
    Evaluate pre-trained model using given dataset loader.
    Called on every epoch.
    :param model: Pretrained pytorch model
    :type model: torch.nn.Module
    :param loader: dataloader configured for the epoch.
    :type loader: :class:`torch.utils.data.DataLoader`
    :param criterion: loss function to use
    :type criterion: function
    :return: Dict with "accuracy", "loss" and "total_correct"
    """
    model.eval()
    loss = 0
    total_correct = 0
    with torch.no_grad():
        for data, target in tqdm(loader, leave=False):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            total_correct += pred.eq(target.view_as(pred)).sum().item()
    
    return {"accuracy": total_correct / len(loader.dataset), 
            "loss": loss / len(loader.dataset), 
            "total_correct": total_correct}

### Parameters

In [16]:
# Training parameters
LEARNING_RATE = 0.02
LEARNING_RATE_GAMMA = 0.8
MOMENTUM = 0.0
EPOCHS = 15
FIRST_EPOCH_BATCH_SIZE = 4
TRAIN_BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000

### Create Sparse MNIST model

There are 2 ways to create **nupic.torch** sparse models. You can import the models from **nupic.torch.models** or use pytorch's [torch.hub](https://pytorch.org/docs/stable/hub.html) API.

In this example we will import the models. 

In [17]:
from nupic.torch.models import MNISTSparseCNN
# For this example we will use the default values. 
# See MNISTSparseCNN documentation for all possible parameters and their values.
model = MNISTSparseCNN().to(device)
print(model)

MNISTSparseCNN(
  (cnn1): SparseWeights2d(
    sparsity=0.4
    (module): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  )
  (cnn1_maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn1_kwinner): KWinners2d(channels=32, local=False, break_ties=False, n=0, percent_on=0.1, boost_strength=1.5, boost_strength_factor=0.85, k_inference_factor=1.0, duty_cycle_period=1000)
  (cnn2): SparseWeights2d(
    sparsity=0.55
    (module): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  )
  (cnn2_maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2_kwinner): KWinners2d(channels=64, local=False, break_ties=False, n=0, percent_on=0.2, boost_strength=1.5, boost_strength_factor=0.85, k_inference_factor=1.0, duty_cycle_period=1000)
  (flatten): Flatten()
  (linear): SparseWeights(
    sparsity=0.8
    (module): Linear(in_features=1024, out_features=700, bias=True)
  )
  (linear_kwinner): KWinners(n=700, percent_on=0.2, boo

### Load MNIST Dataset

In [18]:
normalize = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('data', train=True, download=True, transform=normalize)
test_dataset = datasets.MNIST('data', train=False, transform=normalize)

# Configure data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True)
first_loader = torch.utils.data.DataLoader(train_dataset, batch_size=FIRST_EPOCH_BATCH_SIZE, shuffle=True)

### Train
On the first epoch we use smaller batch size to calculate the duty cycles used by the k-winner function. Once the duty cycles stabilize we can use larger batch sizes. Using the `post_batch`, we rezero the weights after every batch to keep the initial sparsity constant.

In [19]:
from nupic.torch.modules import rezero_weights, update_boost_strength

def post_batch(model):
    model.apply(rezero_weights)

sgd = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
lr_scheduler = optim.lr_scheduler.StepLR(sgd, step_size=1, gamma=LEARNING_RATE_GAMMA)
train(model=model, loader=first_loader, optimizer=sgd, criterion=F.nll_loss, post_batch_callback=post_batch)
lr_scheduler.step()

  0%|          | 0/15000 [00:00<?, ?it/s]

After each we apply the boost strength factor

In [20]:
%%capture
model.apply(update_boost_strength)

Test and print results

In [21]:
test(model=model, loader=test_loader, criterion=F.nll_loss)

  0%|          | 0/10 [00:00<?, ?it/s]

{'accuracy': 0.9774, 'loss': 0.06903478813171386, 'total_correct': 9774}

At this point the duty cycles should be stable and we can train on larger batch sizes

In [22]:
for epoch in range(1, EPOCHS):
    train(model=model, loader=train_loader, optimizer=sgd, criterion=F.nll_loss, post_batch_callback=post_batch)
    lr_scheduler.step()
    model.apply(rezero_weights)
    model.apply(update_boost_strength)
    results = test(model=model, loader=test_loader, criterion=F.nll_loss)
    print(results)

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'accuracy': 0.9906, 'loss': 0.03069741744995117, 'total_correct': 9906}


  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

{'accuracy': 0.9921, 'loss': 0.02723951950073242, 'total_correct': 9921}


  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'accuracy': 0.9915, 'loss': 0.027270044326782227, 'total_correct': 9915}


### Noise
Add noise to the input and check the test accuracy

In [23]:
class RandomNoise(object):
    """
    An image transform that adds noise to random pixels in the image.
    """
    def __init__(self, noise_level=0.0, white_value=0.1307 + 2*0.3081):
        """
        :param noise_level:
          From 0 to 1. For each pixel, set its value to white_value with this
          probability. Suggested white_value is 'mean + 2*stdev'
        """
        self.noise_level = noise_level
        self.white_value = white_value

    def __call__(self, image):
        a = image.view(-1)
        num_noise_bits = int(a.shape[0] * self.noise_level)
        noise = np.random.permutation(a.shape[0])[0:num_noise_bits]
        a[noise] = self.white_value
        return image

In [24]:
noise_score = 0
for noise in [0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
    noise_transform = transforms.Compose([transforms.ToTensor(), RandomNoise(noise), 
                                      transforms.Normalize((0.1307,), (0.3081,))])
    noise_dataset = datasets.MNIST('data', train=False, transform=noise_transform)
    noise_loader = torch.utils.data.DataLoader(noise_dataset, 
                                               batch_size=TEST_BATCH_SIZE, 
                                               shuffle=True)

    results = test(model=model, loader=noise_loader, criterion=F.nll_loss)
    noise_score += results["total_correct"]
    print(noise, ":", results)

  0%|          | 0/10 [00:00<?, ?it/s]

0.0 : {'accuracy': 0.9915, 'loss': 0.027270044326782227, 'total_correct': 9915}


  0%|          | 0/10 [00:00<?, ?it/s]

0.05 : {'accuracy': 0.9897, 'loss': 0.032641384887695316, 'total_correct': 9897}


  0%|          | 0/10 [00:00<?, ?it/s]

...

0.4 : {'accuracy': 0.9365, 'loss': 0.20168538818359374, 'total_correct': 9365}


  0%|          | 0/10 [00:00<?, ?it/s]

0.45 : {'accuracy': 0.9088, 'loss': 0.2825202178955078, 'total_correct': 9088}


  0%|          | 0/10 [00:00<?, ?it/s]

0.5 : {'accuracy': 0.8725, 'loss': 0.4023683654785156, 'total_correct': 8725}


In [14]:
print("noise_score:", noise_score)

noise_score: 103228
